## Задача

Для реализации задачи бизнесу потребовался функционал определения языка текста.

Необходимо предложить архитектуру нейросети для решения этой задачи в виде кода модели TensorFlow/Keras и описания деталей процесса обучения — выбор функции потерь, оптимизатора и проч. Будет здорово так же получить объяснение выбранной архитектуры и описание её преимуществ / недостатков.

## Решение

Для решения задачи бизнеса нет необходимости создавать исключительно свои решения. Возможно, уже есть готовые решения для данной задачи, тк для обучения нейросети с нуля необходимо собрать достаточно большую обучающую выборку. Безусловно, википедия отлично с этим мправится, но не стоит забывать о временных и вычислительных затрат.

Как пример готового решения есть библиотека [langdetect](https://pypi.org/project/langdetect/), которая может определять язык текста. 

Обучение проводилось на текстах википедии и применялась naive Bayesian filter для определения языка. Судя по описанию библиотека позволяет детектировать 53 языка с 99% точностью, а также есть возможность добавлять свои языки.

Также вариантом можно взять предобученные эмбединги из бертоподобных языковых моделей и обучить многоклассовый логрег на них. Либо же получить эмбединги за счет самостоятельного обучения языковой модели на основе берт архтектуры, что достаточно затратно. В целом, если можно решить задачу детекции языка наивным баесом с нормальным качеством, то обучать что-то сложное/тяжелое не имеет большого смысла.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

In [ ]:
VOCAB_SIZE = ...
SEQ_MAX_LEN = ...
NUM_LANGUAGES = ...

In [ ]:
model = keras.models.Sequential(
        [
            layers.Embedding(input_dim=VOCAB_SIZE, output_dim=100, input_length=SEQ_MAX_LEN),
            layers.Conv1D(filters=64, kernel_size=5, padding='same', activation="relu"),
            layers.MaxPooling1D(pool_size=5, padding="same"),
            layers.Flatten(),
            layers.Dense(NUM_LANGUAGES, activation="softmax"),
        ]
    )

В целом можно было бы использовать только Dense слои для данной задачи, но сверточный слой позволяет сократить кол-во параметров для обучения, а также позволит "смотреть" на контекст в тексте.

*Что можно было бы здесь улучшить?*

Потюнить размерности в зависимости от кол-ва данных, либо же увеличить кол-во сверточных слоев.
Также улучшением было бы решение взять другую архитектуру в целом.

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

- **sparse_categorical_crossentropy**- у нас многоклассовая классификация с таргетом конкретного класса, данный лосс из коробки отлично помодет разделить классы. Однако, если у нас будет достаточно большой дисбаланс в выборке, те языки будут представлены в размном объеме, то необходимо будет заменить лосс. Первоначально можно будет попробовать испрльзовать focal loss, но, вероятно, надо будет поэксперименировать с этим параметром.
- **adam** - самый распространенно используемый оптимизатор. Единственно, что можно было бы здесь поправить так это настроить learning rate, сделать его не постоянным, а изменяемым во времени обучения.

In [ ]:
es = EarlyStopping(monitor="val_loss", patience=2)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=[es])